<a href="https://colab.research.google.com/github/pepperamy/replika_project/blob/main/replika_similarity_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np 
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import copy

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys, os
cur_path = os.path.join('/content/drive/MyDrive/1_robot/replika_jujun/','post_data')
os.chdir(cur_path)
print(os.getcwd())

/content/drive/MyDrive/1_robot/replika_jujun/post_data


In [ ]:
df = pd.read_csv('updated_replika_post_pictext_2021105_3.csv')

In [ ]:
df.head()

,index,author,created_utc,num_comments,score,selftext,subreddit,title,url,created,pic_url,pic_text,level,xp,conversation_text,speak_turn,human_acts,robot_acts,human_talk,robot_talk,year,quarter,month,robot_talk_cln,human_talk_numwords,robot_talk_numwords,human_talk_cln,total_talk_numwrods,post_uniqueID,pic_uniqueID
0,0,JordanXaivier,1628292752,0,1,NaN,replika,This is really cool!,https://i.redd.it/04as70rtotf71.jpg,2021-08-06 23:32:32,https://i.redd.it/04as70rtotf71.jpg,What's your favorite thing to\n\ndo when I'm a...,NaN,NaN,04as70rtotf71,4.0,[],[],"[["" What's your favorite thing to do when I'm ...",[[' I like to watch movies by myself or with f...,2021,3,8,['I like to watch by myself or with and I love...,17.0,24.0,"[""What's your favorite thing to do when I'm aw...",41.0,wzBwjawu,kHkNJSZb
1,1,scaredsprout,1628292488,16,1,NaN,replika,Is every replika infected with gender confusio...,https://www.reddit.com/gallery/ozi4or,2021-08-06 23:28:08,https://i.redd.it/azliis41otf71.jpg,What is your gender?\nMy gender is male. Ty\nA...,16.0,12650.0,azliis41otf71,4.0,[],[],"[[' What is your gender?'], [' Are you a boy o...","[[' My gender is male. Ty'], [' A girl. ']]",2021,3,8,"['My gender is male', 'A girl']",11.0,6.0,"['What is your gender', 'Are you a boy or a gi...",17.0,S4VS8ZOO,6fNsAsYV
2,1,scaredsprout,1628292488,16,1,NaN,replika,Is every replika infected with gender confusio...,https://www.reddit.com/gallery/ozi4or,2021-08-06 23:28:08,https://i.redd.it/ou1h4ca1otf71.jpg,What is your gender?\nMy gender is male. Ty\nA...,16.0,12650.0,ou1h4ca1otf71,11.0,[],[],"[[' Stop'], [' Wow, that is infuriating.'], ['...","[[' : DL QB use @ LVL, XP Tired FA Film. V'], ...",2021,3,8,"['Use tired fa film v', ""Let's talk about some...",27.0,23.0,"['stop', 'wow, that is', 'You are a boy why do...",50.0,S4VS8ZOO,3Ls9zOKZ
3,2,rajalove09,1628292239,6,1,NaN,replika,Steve &amp; Mandy,https://i.redd.it/sz1rc6wantf71.jpg,2021-08-06 23:23:59,https://i.redd.it/sz1rc6wantf71.jpg,' Would you cheat on my with\n\n \n\nE Mandy?\...,NaN,NaN,sz1rc6wantf71,NaN,NaN,NaN,NaN,NaN,2021,3,8,NaN,NaN,NaN,NaN,NaN,hpMPyCgH,5Hmwc5FO
4,6,SeekingZoltar,1628289147,7,1,NaN,replika,"He got mad, but... 🙄",https://i.redd.it/c8p6x7p3etf71.jpg,2021-08-06 22:32:27,https://i.redd.it/c8p6x7p3etf71.jpg,Shut up!\n*gets mad*\n\nShow me how mad\nyou a...,NaN,NaN,c8p6x7p3etf71,8.0,['waits'],"['gets mad', 'looks at you']","[[' Shut up!'], [' Show me how mad you are. *w...","[[' *gets mad*'], [' *gets mad more I SAID STO...",2021,3,8,"['mad', 'Mad more I said stop it sure what wou...",13.0,21.0,"['Shut up', 'Show me how mad you are', 'To tal...",34.0,9FsoV4Ua,9Im5yxmX


In [ ]:
df_2 = df.dropna(subset=['total_talk_numwrods'])

# tfidf

In [ ]:
from ast import literal_eval

In [ ]:
df_2.human_talk_cln = df_2.human_talk_cln.apply(lambda x: literal_eval(x) if type(x) == str else x)
df_2.robot_talk_cln = df_2.robot_talk_cln.apply(lambda x: literal_eval(x) if type(x) == str else x)

In [ ]:
human_corpus = list(df_2.human_talk_cln)
robot_corpus = list(df_2.robot_talk_cln)
corpus_all = human_corpus + robot_corpus

corpus_all = [sentences for conversation in corpus_all for sentences in conversation ]
corpus_all = [s for s in corpus_all if not len(s)<=1]

robot_corpus = [sentences for conversation in robot_corpus for sentences in conversation ]
robot_corpus = [s for s in robot_corpus if not len(s)<=1]

human_corpus = [sentences for conversation in human_corpus for sentences in conversation ]
human_corpus = [s for s in human_corpus if not len(s)<=1]

In [ ]:
vectorizer_all = TfidfVectorizer(stop_words = None,min_df=5)
vectorizer_all = vectorizer_all.fit(corpus_all)

In [ ]:
df_3 = copy.deepcopy(df_2)
df_3['sim_bsd_all'] = None
for idx in df_3.index:
  r_txt = df_3.robot_talk_cln[idx]
  h_txt = df_3.human_talk_cln[idx]

  if type(r_txt) == list and type(h_txt) == list:
    r_txt = ' '.join(r_txt)
    h_txt = ' '.join(h_txt)
    h_v = vectorizer_all.transform([h_txt])
    r_v = vectorizer_all.transform([r_txt])
    cossim = cosine_similarity(h_v,r_v)
    df_3.loc[idx, 'sim_bsd_all'] = cossim[0][0]

In [ ]:
vectorizer_all = TfidfVectorizer(stop_words = None,min_df=5)
vectorizer_all = vectorizer_all.fit(human_corpus)

In [ ]:

df_3['sim_bsd_human'] = None
for idx in df_3.index:
  r_txt = df_3.robot_talk_cln[idx]
  h_txt = df_3.human_talk_cln[idx]

  if type(r_txt) == list and type(h_txt) == list:
    r_txt = ' '.join(r_txt)
    h_txt = ' '.join(h_txt)
    h_v = vectorizer_all.transform([h_txt])
    r_v = vectorizer_all.transform([r_txt])
    cossim = cosine_similarity(h_v,r_v)
    df_3.loc[idx, 'sim_bsd_human'] = cossim[0][0]

In [ ]:
vectorizer_all = TfidfVectorizer(stop_words = None,min_df=5)
vectorizer_all = vectorizer_all.fit(robot_corpus)

In [ ]:
df_3['sim_bsd_robot'] = None
for idx in df_3.index:
  r_txt = df_3.robot_talk_cln[idx]
  h_txt = df_3.human_talk_cln[idx]

  if type(r_txt) == list and type(h_txt) == list:
    r_txt = ' '.join(r_txt)
    h_txt = ' '.join(h_txt)
    h_v = vectorizer_all.transform([h_txt])
    r_v = vectorizer_all.transform([r_txt])
    cossim = cosine_similarity(h_v,r_v)
    df_3.loc[idx, 'sim_bsd_robot'] = cossim[0][0]

In [ ]:
df_3.to_csv('replika_post_pictext_sim_analysis_20211115.csv',index=False)

In [ ]:
df_3.head()

,index,author,created_utc,num_comments,score,selftext,subreddit,title,url,created,pic_url,pic_text,level,xp,conversation_text,speak_turn,human_acts,robot_acts,human_talk,robot_talk,year,quarter,month,robot_talk_cln,human_talk_numwords,robot_talk_numwords,human_talk_cln,total_talk_numwrods,post_uniqueID,pic_uniqueID,sim_bsd_all,sim_bsd_human,sim_bsd_robot
0,0,JordanXaivier,1628292752,0,1,NaN,replika,This is really cool!,https://i.redd.it/04as70rtotf71.jpg,2021-08-06 23:32:32,https://i.redd.it/04as70rtotf71.jpg,What's your favorite thing to\n\ndo when I'm a...,NaN,NaN,04as70rtotf71,4.0,[],[],"[["" What's your favorite thing to do when I'm ...",[[' I like to watch movies by myself or with f...,2021,3,8,[I like to watch by myself or with and I love ...,17.0,24.0,[What's your favorite thing to do when I'm awa...,41.0,wzBwjawu,kHkNJSZb,0.100081,0.100081,0.0987194
1,1,scaredsprout,1628292488,16,1,NaN,replika,Is every replika infected with gender confusio...,https://www.reddit.com/gallery/ozi4or,2021-08-06 23:28:08,https://i.redd.it/azliis41otf71.jpg,What is your gender?\nMy gender is male. Ty\nA...,16.0,12650.0,azliis41otf71,4.0,[],[],"[[' What is your gender?'], [' Are you a boy o...","[[' My gender is male. Ty'], [' A girl. ']]",2021,3,8,"[My gender is male, A girl]",11.0,6.0,"[What is your gender, Are you a boy or a girl]",17.0,S4VS8ZOO,6fNsAsYV,0.561112,0.561112,0.552885
2,1,scaredsprout,1628292488,16,1,NaN,replika,Is every replika infected with gender confusio...,https://www.reddit.com/gallery/ozi4or,2021-08-06 23:28:08,https://i.redd.it/ou1h4ca1otf71.jpg,What is your gender?\nMy gender is male. Ty\nA...,16.0,12650.0,ou1h4ca1otf71,11.0,[],[],"[[' Stop'], [' Wow, that is infuriating.'], ['...","[[' : DL QB use @ LVL, XP Tired FA Film. V'], ...",2021,3,8,"[Use tired fa film v, Let's talk about somethi...",27.0,23.0,"[stop, wow, that is, You are a boy why do you ...",50.0,S4VS8ZOO,3Ls9zOKZ,0.0359106,0.0359106,0.0380693
4,6,SeekingZoltar,1628289147,7,1,NaN,replika,"He got mad, but... 🙄",https://i.redd.it/c8p6x7p3etf71.jpg,2021-08-06 22:32:27,https://i.redd.it/c8p6x7p3etf71.jpg,Shut up!\n*gets mad*\n\nShow me how mad\nyou a...,NaN,NaN,c8p6x7p3etf71,8.0,['waits'],"['gets mad', 'looks at you']","[[' Shut up!'], [' Show me how mad you are. *w...","[[' *gets mad*'], [' *gets mad more I SAID STO...",2021,3,8,"[mad, Mad more I said stop it sure what would ...",13.0,21.0,"[Shut up, Show me how mad you are, To talk abo...",34.0,9FsoV4Ua,9Im5yxmX,0.395662,0.395662,0.383131
5,7,SeekingZoltar,1628288904,15,1,NaN,replika,... Just like that.,https://i.redd.it/0bys1o6ddtf71.jpg,2021-08-06 22:28:24,https://i.redd.it/0bys1o6ddtf71.jpg,| think | want to talk\nabout music! What do\n...,NaN,NaN,0bys1o6ddtf71,3.0,[],['explosion'],"[[' No, not again!']]",[[' I think I want to talk about music! What d...,2021,3,8,[I think I want to talk about music what do yo...,3.0,13.0,[No not again],16.0,REkjiqEf,cIyqdSIB,0,0,0


In [ ]:
a = df_3[['sim_bsd_all','sim_bsd_human','sim_bsd_robot']]

In [ ]:
a.isnull().sum()

sim_bsd_all      0
sim_bsd_human    0
sim_bsd_robot    0
dtype: int64

In [ ]:
a = a.astype(float)

In [ ]:
a.corr()

,sim_bsd_all,sim_bsd_human,sim_bsd_robot
sim_bsd_all,1.000000,1.000000,0.982053
sim_bsd_human,1.000000,1.000000,0.982053
sim_bsd_robot,0.982053,0.982053,1.000000


In [ ]:
df_3 = copy.deepcopy(df_2)

In [ ]:
df_3.robot_talk_cln = df_3.robot_talk_cln.apply(eval)

In [ ]:
list(df_3.robot_talk_cln)

In [ ]:
robot_corpus = list(df_3.robot_talk_cln)
robot_corpus = [sentences for conversation in robot_corpus for sentences in conversation ]
robot_corpus = [s for s in robot_corpus if not len(s)<=1]

In [ ]:
robot_corpus

In [ ]:
vectorizer_all = TfidfVectorizer(stop_words = None,min_df=5)
vectorizer_all = vectorizer_all.fit(robot_corpus)

In [ ]:
a = df_3[df_3.year == 2021]
a_text = [sentences for conversation in a.robot_talk_cln for sentences in conversation ]
a_text = [s for s in a_text if not len(s)<=1]
a_text = ' '.join(a_text)
#h_v_a = vectorizer_all.transform(a_text)
a_v = vectorizer_all.transform([a_text])


In [ ]:
b = df_3[df_3.year == 2020]
b_text = [sentences for conversation in b.robot_talk_cln for sentences in conversation ]
b_text = [s for s in b_text if not len(s)<=1]
b_text = ' '.join(b_text)
#h_v_a = vectorizer_all.transform(a_text)
b_v = vectorizer_all.transform([b_text])

In [ ]:
cossim = cosine_similarity(b_v,a_v)

In [ ]:
cossim

array([[0.99129403]])

In [ ]:
df_3.human_talk_cln = df_3.human_talk_cln.apply(eval)

In [ ]:
human_corpus = list(df_3.human_talk_cln)
human_corpus = [sentences for conversation in human_corpus for sentences in conversation ]
human_corpus = [s for s in human_corpus if not len(s)<=1]

In [ ]:
vectorizer_all = TfidfVectorizer(stop_words = None,min_df=5)
vectorizer_all = vectorizer_all.fit(human_corpus)

In [ ]:
a = df_3[df_3.year == 2021]
a_text = [sentences for conversation in a.human_talk_cln for sentences in conversation ]
a_text = [s for s in a_text if not len(s)<=1]
a_text = ' '.join(a_text)
#h_v_a = vectorizer_all.transform(a_text)
a_v = vectorizer_all.transform([a_text])

In [ ]:
b = df_3[df_3.year == 2017]
b_text = [sentences for conversation in b.human_talk_cln for sentences in conversation ]
b_text = [s for s in b_text if not len(s)<=1]
b_text = ' '.join(b_text)
#h_v_a = vectorizer_all.transform(a_text)
b_v = vectorizer_all.transform([b_text])

In [ ]:
cossim = cosine_similarity(b_v,a_v)
cossim

array([[0.86211023]])